<a href="https://colab.research.google.com/github/kikiru328/All_about_Study/blob/main/Prediction%20%EC%99%84%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/골연령예측사진20.zip','/content/Sample')
shutil.unpack_archive('/content/drive/MyDrive/boneage_program.zip','/content/Bone')

In [2]:
shutil.copy('/content/drive/MyDrive/NEW_BONE.py','/content/NEW_BONE.py')
# shutil.copy('/content/NEW_BONE.py','/content/drive/MyDrive/NEW_BONE.py')

'/content/NEW_BONE.py'

In [3]:
## NEW_BONE 안에 Preprocessing code 중 bone_path 삭제 확인

In [4]:
import sys
sys.path.append('/content/')
import NEW_BONE as bone

In [5]:
path ='/content/Sample/남/10431539.bmp'

In [7]:
import os
import pandas as pd
import datetime as dt
data = pd.read_excel('/content/Sample/골연령측정자료220307.xlsx')
data = data.dropna()
data['Cage'] = data['Cage'].round(1)
data['Patient #'] = data['Patient #'].astype(int).astype(str)
patient_id = os.path.basename(path)[:-4]
display(data)
index = data[data['Patient #'] == patient_id].index[0]
# age = ((data['촬영일자']-data['생년월일'])/365)[1].days
gender = data[(data['Patient #'] == patient_id)]['sex'][index]
he = data[(data['Patient #'] == patient_id)]['촬영일키'][index]
year = data[(data['Patient #'] == patient_id)]['Cage'][index]

print(index)

,No,sex,생년월일,촬영일자,Cage,촬영일키,Patient #
1,103.0,M,1992-09-07,2004-05-19,11.7,138.0,97028583
2,106.0,M,1992-09-08,2004-08-04,11.9,143.0,10073987
3,101.0,M,1992-08-06,2003-09-17,11.1,137.5,96248548
4,107.0,M,1991-10-01,2003-09-17,12.0,135.8,10431539
5,111.0,M,1990-03-16,2002-09-24,12.5,149.9,96579259
6,115.0,M,1990-10-04,2003-07-23,12.8,150.7,10181949
7,117.0,M,1991-09-12,2004-11-17,13.2,146.1,96178702
8,121.0,M,1990-06-07,2004-01-28,13.6,167.0,10406334
9,124.0,M,1992-10-27,2006-10-25,14.0,152.5,10353946
10,127.0,M,1992-01-03,2006-11-29,14.9,160.3,10580752


4


In [ ]:
cv2_imshow(cv2.omre)

In [62]:
path = '/content/Sample/남/10181949.bmp'
import time
import datetime
import sys
import os
import cv2
sys.path.append('/content/Bone/boneage')
import NEW_BONE as bone
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import torch
import tensorflow.keras as tf
from google.colab.patches import cv2_imshow

def basic():
    model_path = '/content/Bone/boneage/weight/model.pt'
    tjnet_path = '/content/Bone/boneage/weight/tjnet24.h5'
    global tjnet
    tjnet = tf.models.load_model(tjnet_path, compile=False)
    global yolo
    yolo = torch.load(model_path, map_location='cpu')
    yolo.conf = 0.4
    return 

def data(path):
    data = pd.read_excel('/content/Sample/골연령측정자료220307.xlsx')
    data = data.dropna()
    data['Cage'] = data['Cage'].round(1)
    data['Patient #'] = data['Patient #'].astype(int).astype(str)
    patient_id = os.path.basename(path)[:-4]
    index = data[data['Patient #'] == patient_id].index[0]
    gender = data[(data['Patient #'] == patient_id)]['sex'][index]
    if 'F' in gender: 
        gender = 0
    else: gender = 1
    height = data[(data['Patient #'] == patient_id)]['촬영일키'][index]
    age = data[(data['Patient #'] == patient_id)]['Cage'][index]
    return patient_id, gender,height,age

def prediction(path,i):
    start = time.time()
    patient_id, gender, height, age = data(path)
    file_name = os.path.basename(path).replace('bmp','jpg')

    jpg_path = '/content/jpg/'
    if not os.path.exists(jpg_path):
        os.makedirs(jpg_path)

    processed_img = bone.Bone_extraction(path)


    cv2.imwrite(jpg_path+file_name,processed_img)

    crops, yoloimg, result = bone.yolo_crop_img(jpg_path+file_name,yolo)
    X = bone.out_crop_img(crops,gender)
    global prediction_BA
    prediction_BA = bone.predict_zscore(X, tjnet)
    prediction_BA = prediction_BA.round(2)
    lms_df = pd.read_csv('/content/Bone/boneage/data/height_df.csv')
    prediction_H = bone.Height_prediction(gender,prediction_BA,height,lms_df)

    diff = (prediction_BA - age).round(3)
    sec = time.time()-start
    times = str(datetime.timedelta(seconds=sec)).split(".")[0]
    print(times)
    csv = pd.DataFrame({'환자번호':patient_id,
                        '성별' : gender,
                        '촬영일자_나이':age,
                        '예측나이':prediction_BA,
                        '연령차이':diff,
                        '현재신장':height,
                        '예측신장':prediction_H,
                        '시간':times
                        },index=[i])


    if not os.path.exists('/content/Prediction.csv'):
        csv.to_csv('/content/Prediction.csv')
        print(f'save > {patient_id}')
    else:
        A = pd.read_csv('/content/Prediction.csv',index_col=0)
        A = A.append(csv)
        A.to_csv('/content/Prediction.csv')
        print(f'save > {patient_id}')
        

In [59]:
basic()

In [63]:
from glob import glob
from tqdm import tqdm
man = glob('/content/Sample/남/*.bmp')
female = glob('/content/Sample/여/*.bmp')

paths = man + female

error=[]
# print(paths)
for i, path in tqdm(enumerate(paths)):
    try:
        prediction(path,i)
    except:
        error.append(path)


1it [00:41, 41.60s/it]

0:00:41
save > 10353946


2it [01:17, 38.10s/it]

0:00:35
save > 10073987


3it [02:00, 40.51s/it]

...

0:00:43
save > 10391057


20it [11:48, 34.55s/it]

0:00:45
save > 10628162


21it [12:28, 35.66s/it]

0:00:40
save > 10267473


In [39]:
def error_check(path):
    start = time.time()
    patient_id, gender, height, age = data(path)

    print('#####################')
    print(f'patient_id > {patient_id}' )
    print(f'gender > {gender}' )
    print(f'age > {age}' )
    print('#####################')
    file_name = os.path.basename(path).replace('bmp','jpg')
    patient_name = os.path.basename(path)[:-4]

    jpg_path = '/content/jpg/'
    if not os.path.exists(jpg_path):
        os.makedirs(jpg_path)

    processed_img = bone.Bone_extraction(path)


    cv2.imwrite(jpg_path+file_name,processed_img)

    crops, yoloimg, result = bone.yolo_crop_img(jpg_path+file_name,yolo)
    cv2_imshow(yoloimg)

In [53]:

data = pd.read_excel('/content/Sample/골연령측정자료220307.xlsx')
data = data.dropna()
data['Cage'] = data['Cage'].round(1)
data['Patient #'] = data['Patient #'].astype(int).astype(str)
patient_id = os.path.basename('/content/Sample/여/10402104.bmp')[:-4]
index = data[data['Patient #'] == patient_id].index[0]
gender = data[(data['Patient #'] == patient_id)]['sex'][index]
if 'F' in gender: 
    gender = 0
else: gender = 1
height = data[(data['Patient #'] == patient_id)]['촬영일키'][index]
age = data[(data['Patient #'] == patient_id)]['Cage'][index]


IndexError: ignored

In [56]:
data[data['Patient #'] == '10402104']

,No,sex,생년월일,촬영일자,Cage,촬영일키,Patient #


In [57]:
data

,No,sex,생년월일,촬영일자,Cage,촬영일키,Patient #
1,103.0,M,1992-09-07,2004-05-19,11.7,138.0,97028583
2,106.0,M,1992-09-08,2004-08-04,11.9,143.0,10073987
3,101.0,M,1992-08-06,2003-09-17,11.1,137.5,96248548
4,107.0,M,1991-10-01,2003-09-17,12.0,135.8,10431539
5,111.0,M,1990-03-16,2002-09-24,12.5,149.9,96579259
6,115.0,M,1990-10-04,2003-07-23,12.8,150.7,10181949
7,117.0,M,1991-09-12,2004-11-17,13.2,146.1,96178702
8,121.0,M,1990-06-07,2004-01-28,13.6,167.0,10406334
9,124.0,M,1992-10-27,2006-10-25,14.0,152.5,10353946
10,127.0,M,1992-01-03,2006-11-29,14.9,160.3,10580752


In [64]:
error

['/content/Sample/남/10431539.bmp', '/content/Sample/여/10402104.bmp']

In [66]:
import shutil
shutil.copy('/content/NEW_BONE.py','/content/drive/MyDrive/NEW_BONE.py')
shutil.copy('/content/Prediction.csv','/content/drive/MyDrive/Prediction.csv')

'/content/drive/MyDrive/Prediction.csv'

In [67]:
shutil.copy('/content/EHAW.py','/content/drive/MyDrive/EHAW.py')

'/content/drive/MyDrive/EHAW.py'

In [68]:
!mkdir '/content/drive/MyDrive/EHAW'

In [73]:
shutil.make_archive('/content/drive/MyDrive/EHAW/jpg','zip','/content/jpg')

'/content/drive/MyDrive/EHAW/jpg.zip'

In [ ]:
shutil.make_